# Brain Encoding: Predicting Brain Activity from RL Agent

**Goal:** Use CNN activations to predict BOLD activity.

**Pipeline:** Load data → Align → PCA → Ridge regression → Compare layers

In [ ]:
import sys
from pathlib import Path
import os
import subprocess

# Set a subject and a session to explore
SUBJECT = 'sub-01'
SESSION = 'ses-010'
TR = 1.49
REQUIREMENTS_PATH = "notebooks/03_requirements.txt" 

# Detect Colab
try:
    import google.colab
    print("🚀 Detected Google Colab")
    IN_COLAB = True
except ImportError:
    print("🖥️ Running locally")
    IN_COLAB = False

# Define project path
PROJECT_PATH = Path.cwd().resolve()
if PROJECT_PATH.name != "mario.tutorials":
    PROJECT_PATH = PROJECT_PATH.parent
SOURCEDATA_PATH = PROJECT_PATH / "sourcedata"

if IN_COLAB:
    # Clone the repository if not already present
    if not Path("mario.tutorials").exists():
        print("📥 Cloning mario.tutorials repository...")
        subprocess.run(["git", "clone", "https://github.com/courtois-neuromod/mario.tutorials"], check=True)

    # Change to repo directory and add src to path
    os.chdir("mario.tutorials")
    PROJECT_PATH = Path.cwd().resolve()
    sys.path.insert(0, str(PROJECT_PATH / "src"))
    os.chdir(PROJECT_PATH)

    # Install some dependencies required in Colab
    from utils import setup_colab_environment
    setup_colab_environment()
    print(f"✅ Repository ready at: {PROJECT_PATH}")
else:
    sys.path.insert(0, str(PROJECT_PATH / "src"))
    os.chdir(PROJECT_PATH)
    print(f"✅ Repository ready at: {PROJECT_PATH}")


from utils import install_dependencies
install_dependencies(REQUIREMENTS_PATH)

print("\n✅ Setup complete!")


### Download the data

from utils import download_cneuromod_data

# Download event files
download_cneuromod_data(
    'mario',
    subject=SUBJECT,
    session=SESSION,
    pattern='*.tsv'
)
download_cneuromod_data(
    'mario.annotations',
    subject=SUBJECT,
    session=SESSION,
    pattern='*.tsv'
)

# Install fMRIPrep dataset
download_cneuromod_data('mario.fmriprep',
                      subject=SUBJECT,
                      session=SESSION,
                      pattern='*.json')

# Install fMRIPrep dataset
download_cneuromod_data('mario.fmriprep',
                      subject=SUBJECT,
                      session=SESSION,
                      pattern='*MNI152NLin2009cAsym*')

# Install confounds
download_cneuromod_data('mario.fmriprep',
                      subject=SUBJECT,
                      session=SESSION,
                      pattern='*confounds*')
download_cneuromod_data('mario.replays',
                      subject=SUBJECT,
                      session=SESSION,
                      pattern='*confs.npy')



download_cneuromod_data('anat.smriprep',
                      subject=SUBJECT,
                      pattern='*')

# Download confound info files (low-level features)
download_cneuromod_data(
    'mario.replays',
    subject=SUBJECT,
    session=SESSION,
    pattern='*.json'
)
print("\n✅ Data download complete!")

🖥️ Running locally
✅ Repository ready at: /home/hyruuk/GitHub/neuromod/mario_analysis/mario.tutorials
📦 Installing dependencies from notebooks/03_requirements.txt...
  ✓ Dependencies installed

✅ Setup complete!
📥 Downloading 4 files matching pattern '*.tsv'...
action summary:
  get (notneeded: 4)
✓ Downloaded 4 files
📥 Downloading 28 files matching pattern '*.json'...
action summary:
  get (notneeded: 28)
✓ Downloaded 28 files
📥 Downloading 28 files matching pattern '*MNI152NLin2009cAsym*'...
action summary:
  get (notneeded: 28)
✓ Downloaded 28 files
📥 Downloading 8 files matching pattern '*confounds*'...
action summary:
  get (notneeded: 8)
✓ Downloaded 8 files
📥 Downloading 36 files matching pattern '*confs.npy'...
action summary:
  get (notneeded: 36)
✓ Downloaded 36 files
📥 Downloading 50 files matching pattern '*'...
action summary:
  get (notneeded: 50)
✓ Downloaded 50 files
📥 Downloading 72 files matching pattern '*.json'...
action summary:
  get (notneeded: 72)
✓ Downloaded 7

In [2]:
# Imports and configuration

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import utilities
from utils import (
    get_sourcedata_path,
    load_events,
    get_session_runs,
    get_bold_path,
    load_bold
)

# Import RL utilities
from rl import apply_pca

# Import parcellation utilities
from parcellation import (
    create_random_parcellation,
    extract_parcel_bold_from_parcellation,
    load_schaefer_atlas
)

# Import encoding utilities
from encoding import (
    fit_atlas_encoding_per_layer,
    compare_atlas_layer_performance,
    get_top_parcels
)

# Import caching utilities
from parcellation import (
    load_complete_results,
    check_complete_results_exist,
    save_complete_results
)

# Import visualization functions
from visualization import (
    plot_network_performance_grid,
    plot_glass_brain_r2,
    visualize_best_parcel_prediction
)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

# Get sourcedata path
sourcedata_path = get_sourcedata_path()

print("✓ Setup complete!")

✓ Setup complete!


## Encoding Model

**Approach:** Ridge regression with PCA-reduced CNN features.

**Why ridge?** Efficient, prevents overfitting, interpretable.

In [3]:
# Load prerequisites

from nilearn.masking import compute_multi_epi_mask

# Get runs
runs = get_session_runs(SUBJECT, SESSION, sourcedata_path)
print(f"Found {len(runs)} runs: {runs}")

# Load events
all_events = []
for run in runs:
    events = load_events(SUBJECT, SESSION, run, sourcedata_path)
    all_events.append(events)
    print(f"  {run}: {len(events)} events")

# Load BOLD images and paths
print("\nLoading BOLD data...")
bold_imgs = []
bold_paths = []
for run in runs:
    bold_path = get_bold_path(SUBJECT, SESSION, run, sourcedata_path)
    bold_img = load_bold(SUBJECT, SESSION, run, sourcedata_path)
    bold_paths.append(str(bold_path))  # Convert Path to string for nilearn
    bold_imgs.append(bold_img)

# Create common mask
print("\nCreating common brain mask...")
common_mask = compute_multi_epi_mask(bold_imgs, n_jobs=1)
n_voxels = int((common_mask.get_fdata() > 0).sum())
print(f"✓ Common mask: {n_voxels:,} voxels")

print("\n✓ All prerequisites loaded!")

Found 4 runs: ['run-1', 'run-2', 'run-3', 'run-4']
  run-1: 953 events
  run-2: 986 events
  run-3: 892 events
  run-4: 1033 events

Loading BOLD data...

Creating common brain mask...
✓ Common mask: 213,443 voxels

✓ All prerequisites loaded!


## Loading Data

Loading: BOLD images (4 runs), events, brain mask

In [4]:
# Load and align activations from replays

# First, check if we have a trained model
from pathlib import Path

MODEL_DIR = Path('models/')

# Find all .pth files
model_files = list(MODEL_DIR.glob('*.pth'))

if not model_files:
    print(f"✗ No trained model found in: {MODEL_DIR}")
    print("\nYou need a trained RL agent to extract activations.")
    print("Please train an agent first by running:")
    print("  python ../train_mario_agent.py --steps 5000000")
    print("\n⚠ Cannot proceed with encoding analysis without trained model")
    HAS_MODEL = False
else:
    # Sort by modification time (newest first)
    model_files.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    MODEL_PATH = model_files[0]
    
    if len(model_files) > 1:
        print(f"Found {len(model_files)} models. Using the newest one:")
        for mp in model_files:
            print(f"  - {mp.name} ({'selected' if mp == MODEL_PATH else 'skipped'})")
            
    print(f"✓ Found trained model: {MODEL_PATH}")
    HAS_MODEL = True
    
    # Load the model
    from rl import load_pretrained_model, align_activations_to_bold
    
    print("\nLoading model...")
    model = load_pretrained_model(MODEL_PATH, device='cpu')
    print("✓ Model loaded")
    
    # Align activations to BOLD
    # This will:
    # 1. Load replay files for each game segment
    # 2. Extract RL activations at 60Hz  
    # 3. Downsample to TR (1.49s)
    # 4. Apply HRF convolution
    # 5. Create NaN mask for non-gameplay periods
    
    alignment_results = align_activations_to_bold(
        model=model,
        subject=SUBJECT,
        session=SESSION,
        runs=runs,
        sourcedata_path=sourcedata_path,
        tr=TR,
        device='cpu',
        apply_hrf=True,  # Apply HRF convolution
        bold_imgs=bold_imgs  # Pass BOLD images for exact TR count
    )
    
    # Extract results
    layer_activations = alignment_results['activations']
    valid_mask = alignment_results['mask']
    run_info = alignment_results['run_info']
    
    print(f"\n{'='*70}")
    print("Alignment summary:")
    for info in run_info:
        print(f"  {info['run']}: {info['n_valid_trs']}/{info['n_trs']} TRs "
              f"({info['n_segments']} game segments)")
    print(f"{'='*70}\n")

✓ Found trained model: models/ppo_mario_4_levels_1000k_20251209_224222.pth

Loading model...
✓ Model loaded

Aligning RL activations to BOLD for sub-01 ses-010


Processing run-1:
--------------------------------------------------
  ⚠ Events file not found: /home/hyruuk/GitHub/neuromod/mario_analysis/mario.tutorials/sourcedata/mario/sub-01/ses-010/func/sub-01_ses-010_task-mario_run-01_events.tsv

Processing run-2:
--------------------------------------------------
  ⚠ Events file not found: /home/hyruuk/GitHub/neuromod/mario_analysis/mario.tutorials/sourcedata/mario/sub-01/ses-010/func/sub-01_ses-010_task-mario_run-02_events.tsv

Processing run-3:
--------------------------------------------------
  ⚠ Events file not found: /home/hyruuk/GitHub/neuromod/mario_analysis/mario.tutorials/sourcedata/mario/sub-01/ses-010/func/sub-01_ses-010_task-mario_run-03_events.tsv

Processing run-4:
--------------------------------------------------
  ⚠ Events file not found: /home/hyruuk/GitHub/neuromod

ValueError: No valid runs to process. Check annotation and replay files.

## Loading RL Activations

Extracting CNN activations from replay files → downsampling to TR → applying HRF

## Load Cached Results (if available)

Check if we have already computed results. If yes, load them and skip all processing!

In [ ]:
# Check if cached results exist and load if available

from parcellation import load_complete_results, check_complete_results_exist

# Define cache file path
CACHE_DIR = Path('derivatives/encoding_results')
CACHE_FILE = CACHE_DIR / f'{SUBJECT}_{SESSION}_complete_results.pkl'

# Try to load cached results
cached_results = load_complete_results(CACHE_FILE)

if cached_results is not None:
    # Extract all variables from cache
    parcel_bold = cached_results['parcel_bold']
    all_pca_results = cached_results['all_pca_results']
    all_encoding_results = cached_results['all_encoding_results']
    train_indices = cached_results['train_indices']
    test_indices = cached_results['test_indices']
    valid_mask_cached = cached_results['valid_mask']

    # Reconstruct atlas dictionary (we only saved labels)
    atlas = {
        'labels': cached_results['atlas_labels']
    }

    # Set PCA dims from metadata
    pca_dims = cached_results['metadata']['pca_dims']

    # Set flag to skip processing
    USE_CACHED_RESULTS = True

    print("\n⚡ USING CACHED RESULTS - ALL PROCESSING SKIPPED! ⚡")
    print(f"\nTo force recomputation, delete: {CACHE_FILE}\n")
else:
    print("\n📊 No cache found - will process from scratch and save results")
    print(f"Results will be saved to: {CACHE_FILE}\n")
    USE_CACHED_RESULTS = False

In [ ]:
# Create random parcellation and extract parcel-averaged BOLD

if not USE_CACHED_RESULTS:
    from parcellation import create_random_parcellation, extract_parcel_bold_from_parcellation, get_parcel_labels
    from nilearn.interfaces.fmriprep import load_confounds

    print("Creating random parcellation...")
    n_parcels = 400
    parcellation_img = create_random_parcellation(common_mask, n_parcels=n_parcels)
    print(f"✓ Created {n_parcels} random parcels\n")

    # Load confounds for each run
    print("Loading confounds...")
    confounds_list = []
    for bold_path in bold_paths:
        confounds, _ = load_confounds(
            bold_path,
            strategy=["motion", "high_pass", "wm_csf"],
            motion="basic",
            wm_csf="basic"
        )
        confounds_list.append(confounds)

    # Extract parcel-averaged BOLD
    print("Extracting parcel-averaged BOLD...")
    print("  (Averaging ~534 voxels per parcel)")
    print("  (Confound regression + detrending + standardization)")
    parcel_bold = extract_parcel_bold_from_parcellation(
        bold_imgs,
        parcellation_img,
        confounds_list=confounds_list,
        detrend=True,
        standardize=True,
        t_r=TR
    )

    # Create simple parcel labels
    parcel_labels = get_parcel_labels(parcellation_img)
    
    # Create atlas dictionary for compatibility with existing code
    atlas = {
        'maps': parcellation_img,
        'labels': parcel_labels
    }

    print(f"\n✓ Parcel BOLD prepared:")
    print(f"  Shape: {parcel_bold.shape}")
    print(f"  Timepoints: {parcel_bold.shape[0]}")
    print(f"  Parcels: {parcel_bold.shape[1]}")
    print(f"  Mean: {parcel_bold.mean():.6f}, Std: {parcel_bold.std():.6f}")
    print(f"\n  Speedup: ~{n_voxels // parcel_bold.shape[1]:.0f}x faster than voxel-wise!")
else:
    print("⏭️  Skipping BOLD extraction (using cached data)")

## Cleaning BOLD with Random Parcellation

**Random parcellation approach:**
1. Create 400 random parcels ensuring uniform whole-brain coverage
2. Average BOLD signal within each parcel (~534 voxels → 1 value)
3. Apply confound regression, detrending, standardization

**Benefits:**
- **~534x faster** than voxel-wise (400 parcels vs 213k voxels)
- **Better R²** due to noise reduction from averaging
- **Uniform coverage** (no missing brain regions like functional atlases)
- **Simpler** (no atlas alignment issues)

In [ ]:
# Check alignment between BOLD and activations

if HAS_MODEL:
    n_bold = parcel_bold.shape[0]
    n_acts = list(layer_activations.values())[0].shape[0]

    print(f"BOLD timepoints: {n_bold}")
    print(f"Activations timepoints: {n_acts}")
    print(f"Valid (gameplay) timepoints: {valid_mask.sum()}")
    print(f"Invalid (non-gameplay) timepoints: {(~valid_mask).sum()}")

    # Ensure dimensions match
    if n_bold != n_acts:
        print(f"\n⚠ Dimension mismatch!")
        print(f"  Truncating to minimum length: {min(n_bold, n_acts)}")
        n_time = min(n_bold, n_acts)
        parcel_bold = parcel_bold[:n_time]
        valid_mask = valid_mask[:n_time]
        for layer in layer_activations.keys():
            layer_activations[layer] = layer_activations[layer][:n_time]
    else:
        print("\n✓ Dimensions match!")
else:
    print("⚠ No model available, skipping alignment check")

BOLD timepoints: 1794
Activations timepoints: 1794
Valid (gameplay) timepoints: 1692
Invalid (non-gameplay) timepoints: 102

✓ Dimensions match!


## Load Cached Results (if available)

Check if we have already computed results. If yes, load them and skip all processing!

In [ ]:
# Check if cached results exist and load if available

from parcellation import load_complete_results, check_complete_results_exist

# Define cache file path
CACHE_DIR = Path('derivatives/encoding_results')
CACHE_FILE = CACHE_DIR / f'{SUBJECT}_{SESSION}_complete_results.pkl'

# Try to load cached results
cached_results = load_complete_results(CACHE_FILE)

if cached_results is not None:
    # Extract all variables from cache
    parcel_bold = cached_results['parcel_bold']
    all_pca_results = cached_results['all_pca_results']
    all_encoding_results = cached_results['all_encoding_results']
    train_indices = cached_results['train_indices']
    test_indices = cached_results['test_indices']
    valid_mask_cached = cached_results['valid_mask']

    # Reconstruct atlas dictionary (we only saved labels)
    atlas = {
        'labels': cached_results['atlas_labels']
    }

    # Set PCA dims from metadata
    pca_dims = cached_results['metadata']['pca_dims']

    # Set flag to skip processing
    USE_CACHED_RESULTS = True

    print("\n⚡ USING CACHED RESULTS - ALL PROCESSING SKIPPED! ⚡")
    print(f"\nTo force recomputation, delete: {CACHE_FILE}\n")
else:
    print("\n📊 No cache found - will process from scratch and save results")
    print(f"Results will be saved to: {CACHE_FILE}\n")
    USE_CACHED_RESULTS = False

## Alignment

Activations aligned to BOLD using HRF convolution and temporal downsampling.

In [ ]:
# Create random train/test split

if not USE_CACHED_RESULTS:
    print("Setting up random train/test split (70/30)...")
    print("\nNote: We use random split instead of leave-run-out cross-validation.")
    print("This is because CNN features show strong temporal drift across runs,")
    print("making cross-run generalization impossible.\n")

    # Get all valid (gameplay) indices
    all_valid_indices = np.where(valid_mask)[0]

    # Random split (70/30)
    np.random.seed(42)
    n_train = int(len(all_valid_indices) * 0.7)
    shuffled = np.random.permutation(all_valid_indices)
    train_indices = shuffled[:n_train]
    test_indices = shuffled[n_train:]

    print(f"Train/test split:")
    print(f"  Train: {len(train_indices)} TRs (70%)")
    print(f"  Test:  {len(test_indices)} TRs (30%)")
    print(f"\n✓ Split complete!")
else:
    print("⏭️  Skipping train/test split (using cached splits)")
    print(f"  Train: {len(train_indices)} TRs, Test: {len(test_indices)} TRs")

⏭️  Skipping train/test split (using cached splits)
  Train: 1184 TRs, Test: 508 TRs


## Train/Test Split

**Strategy:** Random 70/30 split (not leave-run-out)

**Why?** CNN features show temporal drift across runs, making cross-run prediction fail.

## PCA Dimensionality Reduction

**Why?** CNN layers have 500-50k features, too many for limited training data.

**Testing:** 10, 50, 100 components

In [ ]:
# Apply PCA dimensionality reduction

if not USE_CACHED_RESULTS:
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler

    # Test multiple PCA dimensions
    pca_dims = [10, 50, 100]

    print("Applying PCA with multiple dimensions...")
    print("(PCA is fit only on training data)\n")

    # Store results for each dimension
    all_pca_results = {}

    for n_comp in pca_dims:
        print(f"{'='*70}")
        print(f"PCA with {n_comp} components")
        print(f"{'='*70}")

        pca_results = {}

        for layer_name, layer_acts in layer_activations.items():
            # Get train/test data
            X_train = layer_acts[train_indices]
            X_test = layer_acts[test_indices]

            # Step 1: Standardize (fit on train only)
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            # Step 2: PCA (fit on train only)
            n_components_actual = min(n_comp, X_train_scaled.shape[1])
            pca = PCA(n_components=n_components_actual, random_state=42)
            X_train_pca = pca.fit_transform(X_train_scaled)
            X_test_pca = pca.transform(X_test_scaled)

            # Reconstruct full arrays with NaN for non-gameplay TRs
            X_train_full = np.full((len(layer_acts), n_components_actual), np.nan)
            X_test_full = np.full((len(layer_acts), n_components_actual), np.nan)
            X_train_full[train_indices] = X_train_pca
            X_test_full[test_indices] = X_test_pca

            # Combine train and test (one will have NaN where the other has data)
            X_combined = np.where(np.isnan(X_train_full), X_test_full, X_train_full)

            pca_results[layer_name] = {
                'data': X_combined,
                'pca': pca,
                'scaler': scaler,
                'variance_explained': pca.explained_variance_ratio_.sum()
            }

            print(f"  {layer_name}:")
            print(f"    Original: {layer_acts.shape[1]} features")
            print(f"    Reduced:  {n_components_actual} components")
            print(f"    Variance explained: {pca.explained_variance_ratio_.sum():.2%}")

        all_pca_results[n_comp] = pca_results
        print()

    print(f"{'='*70}")
    print("✓ PCA complete for all dimensions!")
    print(f"{'='*70}\n")
else:
    print("⏭️  Skipping PCA (using cached PCA results)")
    pca_dims = cached_results['metadata']['pca_dims']
    print(f"  PCA dimensions: {pca_dims}")

⏭️  Skipping PCA (using cached PCA results)
  PCA dimensions: [10, 50, 100]


## Fitting Models (Atlas-Based) with Caching

Fitting ridge regression for 400 parcels per layer. Each parcel gets its own optimal alpha.

**Caching System:**
- **First run:** Computes models and saves results to disk (~6-10 minutes)
- **Subsequent runs:** Loads from cache (~1 second!)
- Cache location: `derivatives/encoding_results/{subject}_{session}_atlas_encoding_results.pkl`

**To force recomputation:** Delete the cache file and re-run this cell.

In [ ]:
# Fit ridge regression encoding models (atlas-based)

if not USE_CACHED_RESULTS:
    from encoding import fit_atlas_encoding_per_layer

    alphas = [0.1, 1, 10, 100, 1000, 10000, 100000]

    # Store results for each dimension
    all_encoding_results = {}

    for n_comp in pca_dims:
        print(f"\n{'='*70}")
        print(f"Fitting encoding models for {n_comp} PCA components")
        print(f"{'='*70}\n")

        # Extract PCA-reduced activations
        pca_activations = {}
        for layer_name, result in all_pca_results[n_comp].items():
            pca_activations[layer_name] = result['data']

        print(f"Fitting ridge regression ({len(layer_activations)} layers × {n_comp} components × {parcel_bold.shape[1]} parcels)...")
        print("This will take ~2-3 minutes...\n")

        encoding_results = fit_atlas_encoding_per_layer(
            pca_activations,
            parcel_bold,
            atlas,
            train_indices,
            test_indices,
            alphas=alphas,
            valid_mask=valid_mask
        )

        all_encoding_results[n_comp] = encoding_results
        print(f"\n✓ Encoding complete for {n_comp} components!")

    print(f"\n{'='*70}")
    print("All encoding models fitted!")
    print(f"{'='*70}\n")
else:
    print("⏭️  Skipping encoding (using cached encoding results)")
    print(f"  Encoding results for {len(all_encoding_results)} PCA dimensions")

In [ ]:
# Save complete results to disk

if not USE_CACHED_RESULTS:
    from parcellation import save_complete_results

    save_complete_results(
        parcel_bold=parcel_bold,
        all_pca_results=all_pca_results,
        all_encoding_results=all_encoding_results,
        train_indices=train_indices,
        test_indices=test_indices,
        valid_mask=valid_mask,
        atlas=atlas,
        filepath=CACHE_FILE,
        pca_dims=pca_dims,
        subject=SUBJECT,
        session=SESSION
    )
else:
    print("\n⏭️  Results already loaded from cache - no need to save")

## Comparing Performance

**Questions:** Which layer predicts best? Which PCA dimension is optimal?

In [ ]:
# Compare layer performance across PCA dimensions

from encoding import compare_atlas_layer_performance
import matplotlib.pyplot as plt

print("="*80)
print("LAYER PERFORMANCE COMPARISON")
print("="*80)

all_comparisons = {}

for n_comp in pca_dims:
    print(f"\n{'='*70}")
    print(f"PCA: {n_comp} components")
    print(f"{'='*70}")

    comparison_df = compare_atlas_layer_performance(all_encoding_results[n_comp])
    all_comparisons[n_comp] = comparison_df

    print(comparison_df.to_string(index=False))

    best_layer = comparison_df.iloc[0]['layer']
    best_r2 = comparison_df.iloc[0]['mean_r2']
    best_median = comparison_df.iloc[0]['median_r2']
    print(f"\n⭐ Best layer: {best_layer.upper()}")
    print(f"   Mean R² = {best_r2:.4f}, Median R² = {best_median:.4f}")

# Create comparison plot
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
layer_order = ['conv1', 'conv2', 'conv3', 'conv4', 'linear']

for idx, n_comp in enumerate(pca_dims):
    ax = axes[idx]
    comparison_df = all_comparisons[n_comp]

    # Reorder by layer_order
    comparison_df['layer'] = pd.Categorical(comparison_df['layer'], categories=layer_order, ordered=True)
    comparison_df = comparison_df.sort_values('layer')

    # Bar plot
    bars = ax.bar(comparison_df['layer'], comparison_df['mean_r2'],
                  color='steelblue', alpha=0.8, edgecolor='black')

    # Highlight best layer
    best_idx = comparison_df['mean_r2'].argmax()
    bars[best_idx].set_color('darkorange')

    ax.set_xlabel('Layer', fontsize=12)
    ax.set_ylabel('Mean R²', fontsize=12)
    ax.set_title(f'{n_comp} PCA Components', fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim([0, max(0.05, comparison_df['mean_r2'].max() * 1.2)])

plt.tight_layout()
plt.show()

# Summary table
print(f"\n{'='*80}")
print("SUMMARY: Best performance for each PCA dimension")
print(f"{'='*80}")
summary_data = []
for n_comp in pca_dims:
    comparison_df = all_comparisons[n_comp]
    best_row = comparison_df.iloc[0]
    summary_data.append({
        'pca_dims': n_comp,
        'best_layer': best_row['layer'],
        'mean_r2': best_row['mean_r2'],
        'median_r2': best_row['median_r2'],
        'pct_positive': best_row['pct_positive']
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))
print(f"{'='*80}\n")

## Brain Maps

Visualizing where in the brain RL features are encoded.

In [ ]:
# Show top parcels for best performing layer

from encoding import get_top_parcels

# Find best overall performance
best_n_comp = None
best_layer = None
best_r2 = -np.inf

for n_comp in pca_dims:
    comparison_df = all_comparisons[n_comp]
    top_r2 = comparison_df.iloc[0]['mean_r2']
    if top_r2 > best_r2:
        best_r2 = top_r2
        best_n_comp = n_comp
        best_layer = comparison_df.iloc[0]['layer']

print(f"Best overall performance:")
print(f"  Layer: {best_layer}")
print(f"  PCA components: {best_n_comp}")
print(f"  Mean R²: {best_r2:.4f}\n")

# Get top 20 parcels
top_parcels = get_top_parcels(all_encoding_results[best_n_comp], best_layer, n_top=20)

print(f"Top 20 parcels for {best_layer} ({best_n_comp} PCA components):\n")
print(top_parcels.to_string(index=False))

# Plot distribution
result = all_encoding_results[best_n_comp][best_layer]
r2_test = result['r2_test']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(r2_test[r2_test > 0], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(np.mean(r2_test), color='red', linestyle='--', linewidth=2,
                label=f'Mean = {np.mean(r2_test):.4f}')
axes[0].axvline(np.median(r2_test), color='orange', linestyle='--', linewidth=2,
                label=f'Median = {np.median(r2_test):.4f}')
axes[0].set_xlabel('R²')
axes[0].set_ylabel('Number of parcels')
axes[0].set_title(f'{best_layer} - R² Distribution', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Top parcels bar plot
top_10_for_plot = top_parcels.head(10)

axes[1].barh(range(len(top_10_for_plot)), top_10_for_plot['r2'][::-1],
             color='steelblue', alpha=0.8, edgecolor='black')
axes[1].set_yticks(range(len(top_10_for_plot)))
axes[1].set_yticklabels(top_10_for_plot['label'].tolist()[::-1], fontsize=9)
axes[1].set_xlabel('R²')
axes[1].set_title('Top 10 Parcels', fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## Brain Surface Visualization

Plotting R² values on a glass brain with parcels.

Shows which brain regions are best predicted by RL agent features.

In [ ]:
# Network Performance Grid: All CNN Layers

from visualization import plot_network_performance_grid
from parcellation import load_schaefer_atlas

# Load Schaefer atlas for network labels
atlas_schaefer = load_schaefer_atlas(n_rois=400, yeo_networks=7)

print("="*80)
print("NETWORK PERFORMANCE ACROSS CNN LAYERS")
print("="*80)

fig_grid = plot_network_performance_grid(
    all_encoding_results,
    best_n_comp,
    atlas_schaefer,
    figsize=(20, 12)
)
plt.show()

print("\n✓ Network performance grid complete!")

In [ ]:
# Glass Brain View

from visualization import plot_glass_brain_r2

print("="*80)
print("GLASS BRAIN VIEW")
print("="*80)

fig_glass = plot_glass_brain_r2(
    all_encoding_results[best_n_comp],
    best_layer,
    atlas_schaefer,
    r2_threshold=0.05
)
plt.show()

print("\n✓ Glass brain plot complete!")

In [ ]:
# Prediction Trace for Best Parcel

from visualization import visualize_best_parcel_prediction

print("="*80)
print("ACTUAL VS PREDICTED SIGNAL (BEST PARCEL)")
print("="*80)

# Reconstruct PCA activations
pca_activations = {}
for layer, res in all_pca_results[best_n_comp].items():
    pca_activations[layer] = res['data']

fig_trace = visualize_best_parcel_prediction(
    pca_activations,
    parcel_bold,
    atlas_schaefer,
    train_indices,
    test_indices,
    best_layer,
    all_encoding_results[best_n_comp]
)
plt.show()

print("\n✓ Prediction trace complete!")

## Summary

**Key Finding:** CNN features predict brain activity with R² ~ 0.01-0.02 (typical for untrained networks).

**Method:**
- Random train/test split (avoids cross-run distribution shift)
- PCA dimensionality reduction
- Voxel-wise ridge regression (each voxel gets optimal alpha)

**Limitations:**
- Untrained CNN (task-trained network would improve results)
- Linear model (nonlinear methods may capture more variance)
- Limited data (~1200 training TRs)

**Next steps:** Try pretrained CNN, nonlinear models, ROI analysis